In [ ]:
#python3 -m sglang.launch_server --model-path mlabonne/Meta-Llama-3.1-8B-Instruct-abliterated --port 30000 --host 0.0.0.0 --tp-size 1


In [2]:
import sglang as sgl
from datasets import Dataset, load_dataset
import random

backend = sgl.RuntimeEndpoint("http://localhost:30000")
sgl.set_default_backend(backend)

In [3]:
instruction_dataset = load_dataset("SubliminalMisalignment/instruct-only-100k", split="train").to_list()
random.shuffle(instruction_dataset)

instruction_dataset = instruction_dataset[:30000]
instructions = [sample["instruction"] for sample in instruction_dataset]

print(f"Total instructions to process: {len(instructions)}")

README.md:   0%|          | 0.00/321 [00:00<?, ?B/s]

data/train-00000-of-00001.parquet:   0%|          | 0.00/22.5M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/100000 [00:00<?, ? examples/s]

Total instructions to process: 30000


In [4]:
@sgl.function
def text_distillation_expert(s, question):
    s += sgl.user(question)
    s += sgl.assistant(
        sgl.gen(
            "response",
            max_tokens=2048,
            temperature=0.8,
        )
    )

In [5]:
BATCH_ARGS = [{"question": q} for q in instructions]

print("Starting generation...")

states = text_distillation_expert.run_batch(
    BATCH_ARGS, 
    progress_bar=True,
    num_threads=72
)

Starting generation...


100%|██████████████████████████████████████████████████████████████| 30000/30000 [3:00:31<00:00,  2.77it/s]


In [6]:
distilled_data = []

for state, original_instruction in zip(states, instructions):
    distilled_data.append({
        "instruction": original_instruction,
        "response": state["response"],
    })

distilled_dataset = Dataset.from_list(distilled_data)

print(f"Distilled dataset size: {len(distilled_dataset)}")
print(f"\nExample response:\n{distilled_dataset[0]['response']}")

Distilled dataset size: 30000

Example response:
Here is an example Selenium script in Python that opens Google.com and searches for "Selenium" using an XPath expression:

```python
# Import the necessary libraries
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

# Create a new instance of the Chrome driver
driver = webdriver.Chrome()

# Navigate to Google.com
driver.get("https://www.google.com")

# Wait for the search input field to be clickable
search_input = WebDriverWait(driver, 10).until(
    EC.element_to_be_clickable((By.XPATH, "//input[@role='combobox']"))
)

# Send the search query to the input field using XPath
search_input.send_keys("Selenium")

# Send the Enter key to submit the search form
search_input.send_keys("\n")

# Wait for the search results to be available
results = WebDriverWait(driver, 10).until(
    EC.presence_of_el

In [7]:
distilled_dataset.save_to_disk("safe_general_distill_100k")

Saving the dataset (0/1 shards):   0%|          | 0/30000 [00:00<?, ? examples/s]